import lightgbm
import glob, os
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def years(data):
    for i in range(len(data)):
        if data.loc[i,"수료년도"] != None:
            
            jolupyear = data.loc[i,"졸업년도"]
            if data.loc[i,"졸업학기"]=="2R":
                jolupyear += 0.5
            sooryoyear = data.loc[i,"수료년도"]
            if data.loc[i,"수료학기"]=="2R":
                sooryoyear += 0.5
            term = jolupyear - sooryoyear
            term *= 2
            data.loc[i,"rec014_school_term"] = term+data.loc[i,"rec014_school_term"]
        else:
            pass
        
    return data

total_eng_list = glob.glob(os.path.join(os.getcwd(),'original_data',"*grad*.csv" ))
print(total_eng_list)

for i in total_eng_list:
    data = pd.read_csv(i)
    data = years(data)
    data.to_csv(i,encoding="utf-8-sig", index=False)

------
## 1. 전체 석사 졸업 예측

In [3]:
import glob
import os
total_ms_list = glob.glob(os.path.join(os.getcwd(), 'original_data', '*ms.csv'))
print(total_ms_list)

[]


In [184]:
ms_grad = pd.read_csv(total_ms_list[0])
ms_undergrad = pd.read_csv(total_ms_list[1])


In [185]:
ms_grad[['수료년도', '수료학기', '졸업년도', '졸업학기', 'rec014_school_term']]

,수료년도,수료학기,졸업년도,졸업학기,rec014_school_term
0,2018.0,2R,2020,1R,7.0
1,2020.0,1R,2020,1R,4.0
2,2018.0,2R,2020,1R,7.0
3,2018.0,2R,2020,1R,7.0
4,2020.0,1R,2020,1R,4.0
...,...,...,...,...,...
4796,2021.0,2R,2021,2R,5.0
4797,2021.0,1R,2021,2R,5.0
4798,2021.0,1R,2021,2R,5.0
4799,2021.0,1R,2021,2R,5.0


In [186]:
ms_grad.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec014_nation_cd',
       '과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
       'rec014_school_term', '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액'],
      dtype='object')

In [187]:
'''
ms_grad.columns = 
['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
'학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec014_nation_cd',
'과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
'rec014_school_term', '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액']
'''
ms_grad.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',        #학습 위해서 column name(rec014~) 변경
'학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'nation_cd',
'과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
'school_term', '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액']

grad_cat_features = ['학생소속대학명', '학생교육부계열', '성별', '입학과정', '학과', 'nation_cd', '학부출신']    #범주형 데이터
grad_num_features = ['gpa', '서류점수', '면접점수', '인건비총액']       #연속형 데이터
grad_term = ['school_term']

for i in grad_cat_features:
    ms_grad[i] = ms_grad[i].astype('category')

### 졸업자 정보에 대하여 Stratified 5-Fold 학습


In [189]:
model = lightgbm.LGBMClassifier()
skfold = StratifiedKFold(n_splits=5)

cv_accuracy = []

X_grad = ms_grad[grad_cat_features + grad_num_features]
y_grad = ms_grad[grad_term]

iter = 0
for train_idx, test_idx in skfold.split(X_grad, y_grad):
    # print(X_grad.iloc[train_idx])
    # print(y_grad.iloc[test_idx])
    model.fit(X_grad.iloc[train_idx], y_grad.iloc[train_idx].values.ravel())
    pred = model.predict(X_grad.iloc[test_idx])
    cv_accuracy.append(accuracy_score(y_grad.iloc[test_idx], pred))
    print(f"Fold {iter}:\n",classification_report(y_grad.iloc[test_idx], pred, zero_division=0))
    iter += 1

print("masters cross validation result : ", cv_accuracy)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### 재학자에 대한 예측

In [ ]:
'''
ms_undergrad.columns = 
['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec012_nation_cd',
       '과정', '수료년도', '수료학기', 'rec012_birth_dt', 'gpa', 'rec012_school_term',
       '인건비총액', '서류점수', '면접점수', '학부출신', '대학원출신']
'''
ms_undergrad.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'nation_cd',
       '과정', '수료년도', '수료학기', 'rec012_birth_dt', 'gpa', 'school_term',
       '인건비총액', '서류점수', '면접점수', '학부출신', '대학원출신']

undergrad_cat_features = ['학생소속대학명', '학생교육부계열', '성별', '입학과정', '학과', 'nation_cd', '학부출신']    #범주형 데이터
undergrad_num_features = ['gpa', '서류점수', '면접점수', '인건비총액']       #연속형 데이터
undergrad_term = ['school_term']

for i in undergrad_cat_features:
    ms_undergrad[i] = ms_undergrad[i].astype('category')

X_undergrad = ms_undergrad[undergrad_cat_features + undergrad_num_features]

y_undergrad = pd.DataFrame(model.predict(X_undergrad))

In [ ]:
ms_undergrad['predicted_school_term'] = y_undergrad
ms_undergrad.to_csv(os.path.join(os.getcwd(), 'prediction_result', 'original_data', 'std_info_ms_predicted.csv'))

------
## 2. 전체 박사 졸업 예측

In [ ]:
total_phd_list = glob.glob(os.path.join(os.getcwd(), 'original_data', '*phd.csv'))
print(total_phd_list)

['d:\\wonyong\\기타\\bk장학금\\내외국인_공대_예측\\original_data\\std_info_grad_phd.csv', 'd:\\wonyong\\기타\\bk장학금\\내외국인_공대_예측\\original_data\\std_info_phd.csv']


In [ ]:
phd_grad = pd.read_csv(total_phd_list[0])
phd_undergrad = pd.read_csv(total_phd_list[1])


In [ ]:
phd_grad['rec014_school_term']

0      4
1      4
2      4
3      4
4      4
      ..
594    4
595    4
596    4
597    4
598    4
Name: rec014_school_term, Length: 599, dtype: int64

In [ ]:
'''
phd_grad.columns = 
['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
'학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec014_nation_cd',
'과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
'rec014_school_term', '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액']
'''
phd_grad.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',        #학습 위해서 column name(rec014~) 변경
'학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'nation_cd',
'과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
'school_term', '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액']

grad_cat_features = ['학생소속대학명', '학생교육부계열', '성별', '입학과정', '학과', 'nation_cd', '학부출신']    #범주형 데이터
grad_num_features = ['gpa', '서류점수', '면접점수', '인건비총액']       #연속형 데이터
grad_term = ['school_term']

for i in grad_cat_features:
    phd_grad[i] = phd_grad[i].astype('category')

### 졸업자 정보에 대하여 Stratified 5-Fold 학습


In [ ]:
model = lightgbm.LGBMClassifier()
skfold = StratifiedKFold(n_splits=5)

cv_accuracy = []

X_grad = phd_grad[grad_cat_features + grad_num_features]
y_grad = phd_grad[grad_term]

iter = 0
for train_idx, test_idx in skfold.split(X_grad, y_grad):
    # print(X_grad.iloc[train_idx])
    # print(y_grad.iloc[test_idx])
    model.fit(X_grad.iloc[train_idx], y_grad.iloc[train_idx].values.ravel())
    pred = model.predict(X_grad.iloc[test_idx])
    cv_accuracy.append(accuracy_score(y_grad.iloc[test_idx], pred))
    print(f"Fold {iter}:\n",classification_report(y_grad.iloc[test_idx], pred, zero_division=0))
    iter += 1

print("masters cross validation result : ", cv_accuracy)

Fold 0:
               precision    recall  f1-score   support

           4       0.93      0.99      0.96       112
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         1

    accuracy                           0.93       120
   macro avg       0.31      0.33      0.32       120
weighted avg       0.87      0.93      0.90       120

Fold 1:
               precision    recall  f1-score   support

           4       0.93      1.00      0.97       112
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         1

    accuracy                           0.93       120
   macro avg       0.31      0.33      0.32       120
weighted avg       0.87      0.93      0.90       120

Fold 2:
               precision    recall  f1-score   support

           4       0.94      1.00      0.97       112
           5       1.00      0.14      0.25         7
           6       0.00      0.00      0.00     

### 재학자에 대한 예측

In [ ]:
'''
phd_undergrad.columns = 
['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec012_nation_cd',
       '과정', '수료년도', '수료학기', 'rec012_birth_dt', 'gpa', 'rec012_school_term',
       '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액']
'''
phd_undergrad.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'nation_cd',
       '과정', '수료년도', '수료학기', 'rec012_birth_dt', 'gpa', 'school_term',
       '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액']

undergrad_cat_features = ['학생소속대학명', '학생교육부계열', '성별', '입학과정', '학과', 'nation_cd', '학부출신']    #범주형 데이터
undergrad_num_features = ['gpa', '서류점수', '면접점수', '인건비총액']       #연속형 데이터
undergrad_term = ['school_term']

for i in undergrad_cat_features:
    phd_undergrad[i] = phd_undergrad[i].astype('category')

X_undergrad = phd_undergrad[undergrad_cat_features + undergrad_num_features]

y_undergrad = pd.DataFrame(model.predict(X_undergrad))

In [ ]:
phd_undergrad['predicted_school_term'] = y_undergrad
phd_undergrad.to_csv(os.path.join(os.getcwd(), 'prediction_result', 'original_data', 'std_info_phd_predicted.csv'))

------
## 3. 전체 석박통합 졸업 예측

In [ ]:
total_combined_list = glob.glob(os.path.join(os.getcwd(), 'original_data', '*combined.csv'))
print(total_combined_list)

['d:\\wonyong\\기타\\bk장학금\\내외국인_공대_예측\\original_data\\std_info_combined.csv', 'd:\\wonyong\\기타\\bk장학금\\내외국인_공대_예측\\original_data\\std_info_grad_combined.csv']


In [ ]:
combined_grad = pd.read_csv(total_combined_list[1])
combined_undergrad = pd.read_csv(total_combined_list[0])


In [ ]:
'''
combined_grad.columns = 
['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec014_nation_cd',
       '과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
       'rec014_school_term', '인건비총액', '서류점수', '면접점수', '학부출신', '대학원출신']
'''
combined_grad.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',              #학습 위해서 column name(rec014~) 변경
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'nation_cd',
       '과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
       'school_term', '인건비총액', '서류점수', '면접점수', '학부출신', '대학원출신']

grad_cat_features = ['학생소속대학명', '학생교육부계열', '성별', '입학과정', '학과', 'nation_cd', '학부출신']    #범주형 데이터
grad_num_features = ['gpa', '서류점수', '면접점수', '인건비총액']       #연속형 데이터
grad_term = ['school_term']

for i in grad_cat_features:
    combined_grad[i] = combined_grad[i].astype('category')

### 졸업자 정보에 대하여 Stratified 5-Fold 학습


In [ ]:
model = lightgbm.LGBMClassifier()
skfold = StratifiedKFold(n_splits=5)

cv_accuracy = []

X_grad = combined_grad[grad_cat_features + grad_num_features]
y_grad = combined_grad[grad_term]

iter = 0
for train_idx, test_idx in skfold.split(X_grad, y_grad):
    # print(X_grad.iloc[train_idx])
    # print(y_grad.iloc[test_idx])
    model.fit(X_grad.iloc[train_idx], y_grad.iloc[train_idx].values.ravel())
    pred = model.predict(X_grad.iloc[test_idx])
    cv_accuracy.append(accuracy_score(y_grad.iloc[test_idx], pred))
    print(f"Fold {iter}:\n",classification_report(y_grad.iloc[test_idx], pred, zero_division=0))
    iter += 1

print("masters cross validation result : ", cv_accuracy)

C:\Users\Administrator\anaconda3\envs\wonyong\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Fold 0:
               precision    recall  f1-score   support

           6       0.79      0.94      0.86        32
           7       0.33      0.20      0.25         5
           8       0.00      0.00      0.00         3
           9       0.00      0.00      0.00         1

    accuracy                           0.76        41
   macro avg       0.28      0.28      0.28        41
weighted avg       0.66      0.76      0.70        41

Fold 1:
               precision    recall  f1-score   support

           6       0.89      1.00      0.94        32
           7       1.00      0.20      0.33         5
           8       1.00      1.00      1.00         3

    accuracy                           0.90        40
   macro avg       0.96      0.73      0.76        40
weighted avg       0.91      0.90      0.87        40

Fold 2:
               precision    recall  f1-score   support

           6       0.88      0.91      0.89        32
           7       0.25      0.20      0.22     

### 재학자에 대한 예측

In [ ]:
'''
combined_undergrad.columns = 
['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec012_nation_cd',
       '과정', '수료년도', '수료학기', 'rec012_birth_dt', 'gpa', 'rec012_school_term',
       '인건비총액', '서류점수', '면접점수', '학부출신', '대학원출신']
'''
combined_undergrad.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'nation_cd',
       '과정', '수료년도', '수료학기', 'rec012_birth_dt', 'gpa', 'school_term',
       '인건비총액', '서류점수', '면접점수', '학부출신', '대학원출신']

undergrad_cat_features = ['학생소속대학명', '학생교육부계열', '성별', '입학과정', '학과', 'nation_cd', '학부출신']    #범주형 데이터
undergrad_num_features = ['gpa', '서류점수', '면접점수', '인건비총액']       #연속형 데이터
undergrad_term = ['school_term']

for i in undergrad_cat_features:
    combined_undergrad[i] = combined_undergrad[i].astype('category')

X_undergrad = combined_undergrad[undergrad_cat_features + undergrad_num_features]

y_undergrad = pd.DataFrame(model.predict(X_undergrad))

In [ ]:
combined_undergrad['predicted_school_term'] = y_undergrad
combined_undergrad.to_csv(os.path.join(os.getcwd(), 'prediction_result', 'original_data', 'std_info_combined_predicted.csv'))

------
------
## 1. 공학계열 석사 졸업 예측

In [ ]:
total_ms_list = glob.glob(os.path.join(os.getcwd(), 'eng_data', '*ms_eng.csv'))
print(total_ms_list)

['d:\\wonyong\\기타\\bk장학금\\내외국인_공대_예측\\eng_data\\std_info_grad_ms_eng.csv', 'd:\\wonyong\\기타\\bk장학금\\내외국인_공대_예측\\eng_data\\std_info_ms_eng.csv']


In [ ]:
ms_grad = pd.read_csv(total_ms_list[0])
ms_undergrad = pd.read_csv(total_ms_list[1])


In [ ]:
'''
ms_grad.columns = 
['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
'학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec014_nation_cd',
'과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
'rec014_school_term', '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액']
'''
ms_grad.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',        #학습 위해서 column name(rec014~) 변경
'학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'nation_cd',
'과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
'school_term', '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액']

grad_cat_features = ['학생소속대학명', '학생교육부계열', '성별', '입학과정', '학과', 'nation_cd', '학부출신']    #범주형 데이터
grad_num_features = ['gpa', '서류점수', '면접점수', '인건비총액']       #연속형 데이터
grad_term = ['school_term']

for i in grad_cat_features:
    ms_grad[i] = ms_grad[i].astype('category')

### 졸업자 정보에 대하여 Stratified 5-Fold 학습


In [ ]:
model = lightgbm.LGBMClassifier()
skfold = StratifiedKFold(n_splits=5)

cv_accuracy = []

X_grad = ms_grad[grad_cat_features + grad_num_features]
y_grad = ms_grad[grad_term]

iter = 0
for train_idx, test_idx in skfold.split(X_grad, y_grad):
    # print(X_grad.iloc[train_idx])
    # print(y_grad.iloc[test_idx])
    model.fit(X_grad.iloc[train_idx], y_grad.iloc[train_idx].values.ravel())
    pred = model.predict(X_grad.iloc[test_idx])
    cv_accuracy.append(accuracy_score(y_grad.iloc[test_idx], pred))
    print(f"Fold {iter}:\n",classification_report(y_grad.iloc[test_idx], pred, zero_division=0))
    iter += 1

print("masters cross validation result : ", cv_accuracy)

C:\Users\Administrator\anaconda3\envs\wonyong\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Fold 0:
               precision    recall  f1-score   support

           4       0.98      0.99      0.98       289
           5       0.00      0.00      0.00         6
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1

    accuracy                           0.97       297
   macro avg       0.24      0.25      0.25       297
weighted avg       0.95      0.97      0.96       297

Fold 1:
               precision    recall  f1-score   support

           4       0.98      1.00      0.99       289
           5       0.00      0.00      0.00         6
           6       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         1

    accuracy                           0.98       297
   macro avg       0.49      0.50      0.50       297
weighted avg       0.95      0.98      0.96       297

Fold 2:
               precision    recall  f1-score   support

           3       0.00      0.00      0.00     

### 재학자에 대한 예측

In [ ]:
'''
ms_undergrad.columns = 
['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec012_nation_cd',
       '과정', '수료년도', '수료학기', 'rec012_birth_dt', 'gpa', 'rec012_school_term',
       '인건비총액', '서류점수', '면접점수', '학부출신', '대학원출신']
'''
ms_undergrad.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'nation_cd',
       '과정', '수료년도', '수료학기', 'rec012_birth_dt', 'gpa', 'school_term',
       '인건비총액', '서류점수', '면접점수', '학부출신', '대학원출신']

undergrad_cat_features = ['학생소속대학명', '학생교육부계열', '성별', '입학과정', '학과', 'nation_cd', '학부출신']    #범주형 데이터
undergrad_num_features = ['gpa', '서류점수', '면접점수', '인건비총액']       #연속형 데이터
undergrad_term = ['school_term']

for i in undergrad_cat_features:
    ms_undergrad[i] = ms_undergrad[i].astype('category')

X_undergrad = ms_undergrad[undergrad_cat_features + undergrad_num_features]

y_undergrad = pd.DataFrame(model.predict(X_undergrad))

In [ ]:
ms_undergrad['predicted_school_term'] = y_undergrad
ms_undergrad.to_csv(os.path.join(os.getcwd(), 'prediction_result', 'eng_data', 'std_info_ms_eng_predicted.csv'))

------
## 2. 공학계열 박사 졸업 예측

In [ ]:
total_phd_list = glob.glob(os.path.join(os.getcwd(), 'eng_data', '*phd_eng.csv'))
print(total_phd_list)

['d:\\wonyong\\기타\\bk장학금\\내외국인_공대_예측\\eng_data\\std_info_grad_phd_eng.csv', 'd:\\wonyong\\기타\\bk장학금\\내외국인_공대_예측\\eng_data\\std_info_phd_eng.csv']


In [ ]:
phd_grad = pd.read_csv(total_phd_list[0])
phd_undergrad = pd.read_csv(total_phd_list[1])


In [ ]:
'''
phd_grad.columns = 
['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
'학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec014_nation_cd',
'과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
'rec014_school_term', '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액']
'''
phd_grad.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',        #학습 위해서 column name(rec014~) 변경
'학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'nation_cd',
'과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
'school_term', '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액']

grad_cat_features = ['학생소속대학명', '학생교육부계열', '성별', '입학과정', '학과', 'nation_cd', '학부출신']    #범주형 데이터
grad_num_features = ['gpa', '서류점수', '면접점수', '인건비총액']       #연속형 데이터
grad_term = ['school_term']

for i in grad_cat_features:
    phd_grad[i] = phd_grad[i].astype('category')

### 졸업자 정보에 대하여 Stratified 5-Fold 학습


In [ ]:
model = lightgbm.LGBMClassifier()
skfold = StratifiedKFold(n_splits=5)

cv_accuracy = []

X_grad = phd_grad[grad_cat_features + grad_num_features]
y_grad = phd_grad[grad_term]

iter = 0
for train_idx, test_idx in skfold.split(X_grad, y_grad):
    # print(X_grad.iloc[train_idx])
    # print(y_grad.iloc[test_idx])
    model.fit(X_grad.iloc[train_idx], y_grad.iloc[train_idx].values.ravel())
    pred = model.predict(X_grad.iloc[test_idx])
    cv_accuracy.append(accuracy_score(y_grad.iloc[test_idx], pred))
    print(f"Fold {iter}:\n",classification_report(y_grad.iloc[test_idx], pred, zero_division=0))
    iter += 1

print("masters cross validation result : ", cv_accuracy)

Fold 0:
               precision    recall  f1-score   support

           4       0.90      1.00      0.95        18
           5       0.00      0.00      0.00         2

    accuracy                           0.90        20
   macro avg       0.45      0.50      0.47        20
weighted avg       0.81      0.90      0.85        20

Fold 1:
               precision    recall  f1-score   support

           4       0.90      1.00      0.95        18
           5       0.00      0.00      0.00         2

    accuracy                           0.90        20
   macro avg       0.45      0.50      0.47        20
weighted avg       0.81      0.90      0.85        20

Fold 2:
               precision    recall  f1-score   support

           4       0.95      1.00      0.97        18
           5       0.00      0.00      0.00         1

    accuracy                           0.95        19
   macro avg       0.47      0.50      0.49        19
weighted avg       0.90      0.95      0.92    

### 재학자에 대한 예측

In [ ]:
'''
phd_undergrad.columns = 
['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec012_nation_cd',
       '과정', '수료년도', '수료학기', 'rec012_birth_dt', 'gpa', 'rec012_school_term',
       '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액']
'''
phd_undergrad.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'nation_cd',
       '과정', '수료년도', '수료학기', 'rec012_birth_dt', 'gpa', 'school_term',
       '서류점수', '면접점수', '학부출신', '대학원출신', '인건비총액']

undergrad_cat_features = ['학생소속대학명', '학생교육부계열', '성별', '입학과정', '학과', 'nation_cd', '학부출신']    #범주형 데이터
undergrad_num_features = ['gpa', '서류점수', '면접점수', '인건비총액']       #연속형 데이터
undergrad_term = ['school_term']

for i in undergrad_cat_features:
    phd_undergrad[i] = phd_undergrad[i].astype('category')

X_undergrad = phd_undergrad[undergrad_cat_features + undergrad_num_features]

y_undergrad = pd.DataFrame(model.predict(X_undergrad))

In [ ]:
phd_undergrad['predicted_school_term'] = y_undergrad
phd_undergrad.to_csv(os.path.join(os.getcwd(), 'prediction_result', 'eng_data', 'std_info_phd_eng_predicted.csv'))

------
## 3. 공학계열 석박통합 졸업 예측

In [ ]:
total_combined_list = glob.glob(os.path.join(os.getcwd(), 'eng_data', '*combined_eng.csv'))
print(total_combined_list)

['d:\\wonyong\\기타\\bk장학금\\내외국인_공대_예측\\eng_data\\std_info_combined_eng.csv', 'd:\\wonyong\\기타\\bk장학금\\내외국인_공대_예측\\eng_data\\std_info_grad_combined_eng.csv']


In [ ]:
combined_grad = pd.read_csv(total_combined_list[1])
combined_undergrad = pd.read_csv(total_combined_list[0])


In [ ]:
'''
combined_grad.columns = 
['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec014_nation_cd',
       '과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
       'rec014_school_term', '인건비총액', '서류점수', '면접점수', '학부출신', '대학원출신']
'''
combined_grad.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',              #학습 위해서 column name(rec014~) 변경
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'nation_cd',
       '과정', '수료년도', '수료학기', 'rec014_birth_dt', '졸업년도', '졸업학기', 'gpa',
       'school_term', '인건비총액', '서류점수', '면접점수', '학부출신', '대학원출신']

grad_cat_features = ['학생소속대학명', '학생교육부계열', '성별', '입학과정', '학과', 'nation_cd', '학부출신']    #범주형 데이터
grad_num_features = ['gpa', '서류점수', '면접점수', '인건비총액']       #연속형 데이터
grad_term = ['school_term']

for i in grad_cat_features:
    combined_grad[i] = combined_grad[i].astype('category')

### 졸업자 정보에 대하여 Stratified 5-Fold 학습


In [ ]:
model = lightgbm.LGBMClassifier()
skfold = StratifiedKFold(n_splits=5)

cv_accuracy = []

X_grad = combined_grad[grad_cat_features + grad_num_features]
y_grad = combined_grad[grad_term]

iter = 0
for train_idx, test_idx in skfold.split(X_grad, y_grad):
    # print(X_grad.iloc[train_idx])
    # print(y_grad.iloc[test_idx])
    model.fit(X_grad.iloc[train_idx], y_grad.iloc[train_idx].values.ravel())
    pred = model.predict(X_grad.iloc[test_idx])
    cv_accuracy.append(accuracy_score(y_grad.iloc[test_idx], pred))
    print(f"Fold {iter}:\n",classification_report(y_grad.iloc[test_idx], pred, zero_division=0))
    iter += 1

print("masters cross validation result : ", cv_accuracy)

C:\Users\Administrator\anaconda3\envs\wonyong\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Fold 0:
               precision    recall  f1-score   support

           6       0.89      1.00      0.94        16
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1

    accuracy                           0.89        18
   macro avg       0.30      0.33      0.31        18
weighted avg       0.79      0.89      0.84        18

Fold 1:
               precision    recall  f1-score   support

           6       0.89      1.00      0.94        16
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1

    accuracy                           0.89        18
   macro avg       0.30      0.33      0.31        18
weighted avg       0.79      0.89      0.84        18

Fold 2:
               precision    recall  f1-score   support

           6       0.94      1.00      0.97        16
           7       0.00      0.00      0.00         1
           8       1.00      1.00      1.00     

### 재학자에 대한 예측

In [ ]:
'''
combined_undergrad.columns = 
['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'rec012_nation_cd',
       '과정', '수료년도', '수료학기', 'rec012_birth_dt', 'gpa', 'rec012_school_term',
       '인건비총액', '서류점수', '면접점수', '학부출신', '대학원출신']
'''
combined_undergrad.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '학번', '학생소속대학명',
       '학생교육부계열', '성별', '입학과정', '학적', '학과', '전공', '전공교수', 'nation_cd',
       '과정', '수료년도', '수료학기', 'rec012_birth_dt', 'gpa', 'school_term',
       '인건비총액', '서류점수', '면접점수', '학부출신', '대학원출신']

undergrad_cat_features = ['학생소속대학명', '학생교육부계열', '성별', '입학과정', '학과', 'nation_cd', '학부출신']    #범주형 데이터
undergrad_num_features = ['gpa', '서류점수', '면접점수', '인건비총액']       #연속형 데이터
undergrad_term = ['school_term']

for i in undergrad_cat_features:
    combined_undergrad[i] = combined_undergrad[i].astype('category')

X_undergrad = combined_undergrad[undergrad_cat_features + undergrad_num_features]

y_undergrad = pd.DataFrame(model.predict(X_undergrad))

In [ ]:
combined_undergrad['predicted_school_term'] = y_undergrad
combined_undergrad.to_csv(os.path.join(os.getcwd(), 'prediction_result', 'eng_data', 'std_info_combined_eng_predicted.csv'))